In [1]:
from bs4 import BeautifulSoup as bs 
import urllib 
import pandas as pd


query_str = 'https://vnexpress.net/category/day/page/{}.html?cateid=1001002&fromdate=1529964000&todate=1531517760&allcate=1001002|1001002|'
#query_str = "https://vnexpress.net/chien-dich-giai-cuu/tag-810752-1.html"

# Get some articles from 'World news' category from 26/6 to 14/7

#### Return a bs object after extracting the articles from the webpage

In [2]:
def get_articles(query):
    return bs(urllib.request.urlopen(query), 'html.parser').find("section", attrs={"class":"sidebar_1"}).find_all("article",attrs={"class":"list_news"})

#### Turn to a data object for processing

In [3]:
def distribute_articles(articles, return_type='df'):      
    article_dict = {'title': [],
             'link': [],
               'description': [],
               'id' : []}
    
    for article in articles: 
        #article_dict['title'].append(article.a.get('title')) # if this does not work, use the one below
        article_dict['title'].append(article.img.get('alt'))
        article_dict['link'].append(article.a.get('href'))
        article_dict['description'].append(article.h4.get_text())
        article_dict['id'].append(article.a.get('href').split('-')[-1][:-5])
    if return_type == 'df':  
        return pd.DataFrame(data=article_dict)
    
    elif return_type == 'dict':
        return article_dict 

#### This is where the real actions begin

result will be the df containing basic info about the articles headlines in vnexpress from 26/6 to 12/7

In [4]:
df_list = [] 

for page in range(1, 21): 
    page_url = query_str.format(str(page))
    articles = get_articles(page_url)
    df_list.append( distribute_articles(articles))
    
result = pd.concat(df_list).reset_index(drop=True)

# Filter news with specified keywords

In [5]:
kws = ['Thái Lan', 'đội bóng', 'Đội bóng', 'hang', 'mắc kẹt', 'thiếu niên']
temp = []
kw_filtered = [] 

for kw in kws: 
    for title in result.title: 
        if kw in title: 
            temp.append(title)
    
    kw_filtered.append(pd.concat([result[result.title == tmp] for tmp in temp])) 
    temp =[ ]#pd.DataFrame(columns = ['title', 'link','description','id']) 

filtered_result = pd.concat(kw_filtered).drop_duplicates().reset_index(drop = True)


In [6]:
filtered_result.link[1]

'https://vnexpress.net/tin-tuc/the-gioi/phan-tich/cuoc-chien-9-ngay-voi-doi-khat-trong-hang-cua-cac-thieu-nien-thai-lan-3777228.html'

In [21]:
for link in filtered_result.link: 
    print(filtered_result.title[filtered_result.link == link])

0    Những thợ lặn thành anh em sau chiến dịch giải...
Name: title, dtype: object
1    Cuộc chiến 9 ngày với đói khát trong hang của ...
Name: title, dtype: object
2    Giải đáp những câu hỏi trong cuộc giải cứu đội...
Name: title, dtype: object
3    Hành trình đưa tin giải cứu đội bóng nhí Thái ...
Name: title, dtype: object
4    Thiếu niên Thái Lan nói không mang theo đồ ăn ...
Name: title, dtype: object
5    Thợ lặn Anh phát hiện đội bóng nhí Thái Lan kh...
Name: title, dtype: object
6    Thái Lan cân nhắc cấp quốc tịch cho 4 thành vi...
Name: title, dtype: object
7    Bác sĩ Australia giải cứu thiếu niên Thái Lan ...
Name: title, dtype: object
8    Cuộc đoàn tụ gia đình qua cửa kính của cậu bé ...
Name: title, dtype: object
9    Lý do các thiếu niên Thái Lan phải nằm cáng kh...
Name: title, dtype: object
10    Trung Quốc giận dữ vì bình luận của Phó thủ tư...
Name: title, dtype: object
11    Các thiếu niên Thái Lan lần đầu xuất hiện sau ...
Name: title, dtype: object
12    Đội bóng

In [33]:
filtered_result.loc[filtered_result.link == link].title.values[0]

'Bác sĩ cấm các thiếu niên được giải cứu đến Nga xem World Cup'

In [35]:
int(filtered_result.id[filtered_result.link == link].values[0])

3775815

## Retrieving data from articles and store them in excel 

Data will be retrived and store in excel with this run

### Testing on one article

In [9]:
def get_article(query):
    return bs(urllib.request.urlopen(query), 'html.parser').find("section", attrs={"class":"sidebar_1"})

In [10]:
test1 = get_article(filtered_result.link[1])

In [11]:
filtered_result.link[1]

'https://vnexpress.net/tin-tuc/the-gioi/phan-tich/cuoc-chien-9-ngay-voi-doi-khat-trong-hang-cua-cac-thieu-nien-thai-lan-3777228.html'

#### How to get the published date: 

In [12]:
test1.span.get_text().split()

['Thứ', 'sáu,', '13/7/2018,', '12:58', '(GMT+7)']

#### How to get the title of this article:

In [14]:
filtered_result.title[2]

'Giải đáp những câu hỏi trong cuộc giải cứu đội bóng nhí Thái Lan'

#### How to get the short overview of this article:

In [12]:
test1.h2.get_text()

'Nước chảy ra từ thạch nhũ trong hang và phương pháp thiền định đã giúp đội bóng Lợn Hoang sống sót trong 9 ngày đầu tiên mắc kẹt.'

#### Core of the article
Note that at the end of it is the name of the author

In [13]:
article_text = ''
for text in test1.find_all('p', attrs={'class':'Normal'}):
    article_text+= text.get_text()

In [17]:
article_text.replace('\t', '').replace(u'\xa0', u' ').replace('\n','')

'12 thiếu niên và huấn luyện viên đội bóng Lợn Hoang hôm 10/7 đã được giải cứu thành công khỏi hang Tham Luang ở tỉnh Chiang Rai, miền bắc Thái Lan sau 18 ngày mắc kẹt, theo BBC.Trước khi được hai thợ lặn người Anh phát hiện đêm 2/7, các cậu bé và huấn luyện viên Ekkapol Chantawong đã làm mọi cách có thể để sống sót trong hang sâu tối tăm, ngập nước suốt 9 ngày, trong đó Ekkapol đóng vai trò rất quan trọng.Truyền thông Thái Lan ban đầu cho rằng đội Lợn Hoang vào hang Tham Luang để tổ chức sinh nhật lần thứ 17 cho Peerapat Sompiangjai, biệt danh Night, nên đã mua đồ ăn nhẹ mang vào. Những gói bánh này được xem là nguồn thực phẩm duy nhất giúp đội bóng cầm cự trong 9 ngày đầu tiên.Tuy nhiên, Ikan Wiboonroongruang, 33 tuổi, mẹ của Chanintr, thành viên nhỏ tuổi nhất trong đội, kể trong cuộc gặp sau cửa kính cách ly ở bệnh viện, con trai nói với cô rằng nhóm chỉ định vào hang chơi một giờ và không mang theo đồ ăn."Sau ba đêm đầu tiên không có thức ăn, con trai tôi đói lả và khóc. Nó chỉ biế

#### Author of this article can also be retrieved by query for 'strong' tag 

In [55]:
test1.strong.get_text()

'Huyền Lê'

#### Getting multi-media links in the article 

Save up for future usage

##### Video

In [81]:
test1.find_all('video')

[<video active-mode="360" ads="https://g.eclick.vn/video.xml?id=3457&amp;url=https://vnexpress.net/tin-tuc/the-gioi/phan-tich/cuoc-chien-9-ngay-voi-doi-khat-trong-hang-cua-cac-thieu-nien-thai-lan-3777228.html" adsconfig="" controls="" data-240="https://v.vnecdn.net/vnexpress/video/web/mp4/240p/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902.mp4" data-360="https://v.vnecdn.net/vnexpress/video/web/mp4/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902.mp4" data-480="" data-720="" data-ex="st=0&amp;bs=0&amp;pt=0" data-src="https://v.vnecdn.net/vnexpress/video/web/mp4/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902.mp4" id="media-video-211027" max-mode="360" playsinline="" preload="auto" src="https://d.vnecdn.net/vnexpress/video/video/web/mp4/,240p,,/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902/vne/master.m3u8" style="width: 100%; height: 100%;" type="application/x-mpegURL" webki

In [77]:
test1.video.attrs

{'id': 'media-video-211027',
 'preload': 'auto',
 'playsinline': '',
 'webkit-playsinline': '',
 'data-src': 'https://v.vnecdn.net/vnexpress/video/web/mp4/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902.mp4',
 'src': 'https://d.vnecdn.net/vnexpress/video/video/web/mp4/,240p,,/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902/vne/master.m3u8',
 'type': 'application/x-mpegURL',
 'controls': '',
 'style': 'width: 100%; height: 100%;',
 'data-240': 'https://v.vnecdn.net/vnexpress/video/web/mp4/240p/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902.mp4',
 'data-360': 'https://v.vnecdn.net/vnexpress/video/web/mp4/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902.mp4',
 'data-480': '',
 'data-720': '',
 'max-mode': '360',
 'active-mode': '360',
 'ads': 'https://g.eclick.vn/video.xml?id=3457&url=https://vnexpress.net/tin-tuc/the-gioi/phan-tich/cuoc-chien-9-ngay-voi-doi-khat-trong-hang-cua-

##### Main src to take for video

In [78]:
test1.video['data-src']

'https://v.vnecdn.net/vnexpress/video/web/mp4/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902.mp4'

##### Images

In [80]:
test1.find_all('img')

[<img alt="Đội bóng Lợn Hoang trong một chuyến dã ngoại trước khi gặp nạn trong hang Tham Luang. Ảnh: Twitter." data-natural-width="500" src="https://vcdn-vnexpress.vnecdn.net/2018/07/13/anh-1-3184-1531458767.jpg"/>,
 <img alt="Các thiếu niên trú ngụ trên một mỏm đá trong hang khi được thợ lặn Anh phát hiện đêm 2/7. Ảnh: Hải quân Thái Lan." data-natural-width="500" src="https://vcdn-vnexpress.vnecdn.net/2018/07/13/anh-2-9044-1531458767.jpg"/>,
 <img alt="Các cầu thủ nhí Thái Lan vẫy tay chào trên gường bệnh " src="https://iv.vnecdn.net/vnexpress/images/web/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902_500x300.jpg"/>,
 <img alt="Đội bóng nhí Thái không tự lặn mà nằm trên cáng trong quá trình giải cứu" src="https://iv.vnecdn.net/vnexpress/images/web/2018/07/12/doi-bong-nhi-thai-khong-tu-lan-ma-nam-tren-cang-trong-qua-tr-1531352483_500x300.jpg"/>]

In [83]:
for image in test1.find_all('img'): 
    print("Image name: {}, src: {}".format(image['alt'], image['src']))

Image name: Đội bóng Lợn Hoang trong một chuyến dã ngoại trước khi gặp nạn trong hang Tham Luang. Ảnh: Twitter., src: https://vcdn-vnexpress.vnecdn.net/2018/07/13/anh-1-3184-1531458767.jpg
Image name: Các thiếu niên trú ngụ trên một mỏm đá trong hang khi được thợ lặn Anh phát hiện đêm 2/7. Ảnh: Hải quân Thái Lan., src: https://vcdn-vnexpress.vnecdn.net/2018/07/13/anh-2-9044-1531458767.jpg
Image name: Các cầu thủ nhí Thái Lan vẫy tay chào trên gường bệnh , src: https://iv.vnecdn.net/vnexpress/images/web/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902_500x300.jpg
Image name: Đội bóng nhí Thái không tự lặn mà nằm trên cáng trong quá trình giải cứu, src: https://iv.vnecdn.net/vnexpress/images/web/2018/07/12/doi-bong-nhi-thai-khong-tu-lan-ma-nam-tren-cang-trong-qua-tr-1531352483_500x300.jpg


### Write these in an Excel file

#### Generate the list of the respective attributes: 
- Author 
- DateTime String 
- Text
- Short synopsis 
- Images-alt 
- Images-link 
- Video_link

In [21]:
def get_article(query):
    return bs(urllib.request.urlopen(query), 'html.parser').find("section", attrs={"class":"sidebar_1"})

class Article(object)  : 
    def __init__(self, article_soup):
        self.article = article_soup
        self.author = self.get_author()
        self.datetime = self.get_datetime() 
        self.text = self.get_text()
        self.synopsis = self.get_synopsis()
        self.imgs = self.get_imgs() #This is a tuple of (alt, src)
        self.videos = self.get_videos()
        self.title = self.article.title[1]
    
    def get_author(self): 
        return self.article.strong.get_text()

    def get_datetime(self): 
        return self.article.span.get_text() 

    def get_text(self):     
        article_text = ''
        for text in self.article.find_all('p', attrs={'class':'Normal'}):
            article_text+= text.get_text()    
        return article_text.replace('\t', '').replace(u'\xa0', u' ').replace('\n','')

    def get_synopsis(self): 
        return self.test.h2.get_text()
    
    def get_imgs(self):
        return {'alt': [img['alt'] for img in self.article.find_all('img')], 'src': [img['src'] for img in self.article.find_all('img')]}
    
    def get_videos(self): 
        return [video['data-src'] for video in self.article.find_all('video')]
    

class Articles(object): 
    def __init__(self, articles_list):
        self.articles_list = articles_list
        
        self.titles = []
        self.links = []
        self.ids = []
        self.authors = []
        self.datetime = []
        self.text = []
        self.synopses = []
        self.img_alts = []
        self.img_links = []
        #current_article
        self.video_links = []
        
    def generate_articles_content(self): 
        
        for link in self.articles_list.link: 
            current_art = Article(get_article(link))
            
            self.titles.append(self.articles_list.title[self.articles_list.link == link].values[0])
            self.links.append(link)
            self.ids.append(self.articles_list.loc[self.articles_list.link == link].values[0])
            self.authors.append(current_article.author)
            self.datetime.append(current_article.datetime)
            self.text.append(current_article.text)
            self.synopses.append(current_article.synopses)
            self.img_alts.append(current_article.imgs['alt'])
            self.img_links.append(current_article.imgs['src'])
            self.video_links.append(current_article.videos)
            
        return len(self.titles) == len(self.links) == len(self.ids) == len(self.authors) == len(self.datetime) == len(self.text) == len(self.synopses) == len(self.img_alts) == len(self.img_links) == len(self.video_links)
            
            
    def generate_articles_content_dict(self):
        
        return {'title': self.titles, #From filtered_result object
                 'link':self.links, #in Articles object 
                 'id': self.ids, #from filtered_result object
                 'author': self.authors, #attr of Article Object 
                 'datetime': self.datetime,
                 'text': self.text,
                 'synopsis': self.synopses,
                 'img_alt': self.img_alts,
                 'img_link': self.img_links,
                 #'video_alt': self.video_alts,
                 'video_link': self.video_links}
    

#### Create a DataFrame for storing articles

Each article has the following attributes: 
- Title (from list of link dataframe)
- Link (from list of link dataframe)
- ID (from list of link dataframe)
- Author 
- DateTime String 
- Text
- Short synopsis 
- Images-alt 
- Images-link 
- Video_link

##### Rewrite everything again with this addition: 

In [7]:
from bs4 import BeautifulSoup as bs 
import urllib 
import pandas as pd


query_str = 'https://vnexpress.net/category/day/page/{}.html?cateid=1001002&fromdate=1529964000&todate=1531517760&allcate=1001002|1001002|'
#query_str = "https://vnexpress.net/chien-dich-giai-cuu/tag-810752-1.html"

def get_articles(query):
    return bs(urllib.request.urlopen(query), 'html.parser').find("section", attrs={"class":"sidebar_1"}).find_all("article",attrs={"class":"list_news"})

def distribute_articles(articles, return_type='df'):      
    article_dict = {'title': [],
             'link': [],
               'description': [],
               'id' : []}
    
    for article in articles: 
        #article_dict['title'].append(article.a.get('title')) # if this does not work, use the one below
        article_dict['title'].append(article.img.get('alt'))
        article_dict['link'].append(article.a.get('href'))
        article_dict['description'].append(article.h4.get_text())
        article_dict['id'].append(article.a.get('href').split('-')[-1][:-5])
    if return_type == 'df':  
        return pd.DataFrame(data=article_dict)
    
    elif return_type == 'dict':
        return article_dict 
    
df_list = [] 

for page in range(1, 23): 
    page_url = query_str.format(str(page))
    articles = get_articles(page_url)
    df_list.append( distribute_articles(articles))
    
result = pd.concat(df_list).reset_index(drop=True)

kws = ['Thái Lan', 'đội bóng', 'Đội bóng', 'hang', 'mắc kẹt', 'thiếu niên']
temp = []
kw_filtered = [] 

for kw in kws: 
    for title in result.title: 
        if kw in title: 
            temp.append(title)
    
    kw_filtered.append(pd.concat([result[result.title == tmp] for tmp in temp])) 
    temp =[ ]#pd.DataFrame(columns = ['title', 'link','description','id']) 

filtered_result = pd.concat(kw_filtered).drop_duplicates().reset_index(drop = True)

In [6]:
filtered_result

,title,link,description,id
0,Những thợ lặn thành anh em sau chiến dịch giải...,https://vnexpress.net/tin-tuc/the-gioi/tu-lieu...,Điều quý giá nhất thợ lặn Douglas Yeo nhận đượ...,3777298
1,Cuộc chiến 9 ngày với đói khát trong hang của ...,https://vnexpress.net/tin-tuc/the-gioi/phan-ti...,Nước chảy ra từ thạch nhũ trong hang và phương...,3777228
2,Giải đáp những câu hỏi trong cuộc giải cứu đội...,https://vnexpress.net/tin-tuc/the-gioi/phan-ti...,Những câu hỏi như tại sao các cậu bé đi sâu đế...,3777195
3,Hành trình đưa tin giải cứu đội bóng nhí Thái ...,https://vnexpress.net/tin-tuc/the-gioi/tu-lieu...,"Bầu không khí ""phi thường"" lúc tất cả thành vi...",3777154
4,Thiếu niên Thái Lan nói không mang theo đồ ăn ...,https://vnexpress.net/tin-tuc/the-gioi/thieu-n...,Cậu bé nhỏ tuổi nhất trong đội bóng mới được g...,3777286
5,Thợ lặn Anh phát hiện đội bóng nhí Thái Lan kh...,https://vnexpress.net/tin-tuc/the-gioi/tho-lan...,John Volanthen không nhận thành tích về mình m...,3777109
6,Thái Lan cân nhắc cấp quốc tịch cho 4 thành vi...,https://vnexpress.net/tin-tuc/the-gioi/thai-la...,"4 người được giải cứu khỏi hang Tham Luang, tr...",3777064
7,Bác sĩ Australia giải cứu thiếu niên Thái Lan ...,https://vnexpress.net/tin-tuc/the-gioi/cuoc-so...,"Nhiều người Australia cho rằng Harris, bác sĩ ...",3776895
8,Cuộc đoàn tụ gia đình qua cửa kính của cậu bé ...,https://vnexpress.net/tin-tuc/the-gioi/cuoc-so...,Cậu bé nhỏ tuổi nhất đội bóng nhí được cứu khỏ...,3776713
9,Lý do các thiếu niên Thái Lan phải nằm cáng kh...,https://vnexpress.net/tin-tuc/the-gioi/tu-lieu...,Việc các em nằm yên trong cáng giúp thợ lặn dễ...,3776602


In [19]:
art_df = {'tiltle': [], #From filtered_result object
         'link':[], #in Articles object 
         'id': [], #from filtered_result object
         'author': [], #attr of Article Object 
         'datetime':[],
         'text': [],
         'synopsis':[],
         'img_alt':[],
         'img_link': [],
         'video_alt': [],
         'video_link': []}

for result in filtered_result: 
    article = Article(result)
    


In [18]:
# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Data': [10, 20, 30, 20, 15, 30, 45]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('pandas_simple.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()